In [1]:
import pandas as pd
import sys

sys.path.insert(0, '/Users/sengineer/Documents/GitHub/PatentsView-Evaluation/pv_evaluation/metrics')
from cluster import cluster_precision_recall

In [2]:
ref = pd.read_csv('data/eval_als.txt', sep='\t', header=None, names=['mention', 'cluster'], dtype={'mention': 'string', 'cluster': 'int16'})
ref['mention'] = ref.apply(lambda x: "US" + x.mention, axis=1)

#convert to series
ref.set_index('mention', inplace=True)
ref_series = ref.iloc[:, 0]
ref_series

mention
US5294443-0       0
US6207855-1       1
US5767288-2       2
US4996481-1       3
US6727070-0       4
               ... 
US5150706-0     493
US7556935-2    1516
US8405044-1    2534
US7088104-0    1517
US6068972-0     528
Name: cluster, Length: 41347, dtype: int16

In [18]:
#loading result and id datasets for merge
pred_result = pd.read_csv('data/autosequence_cleaned.csv', index_col = 0)
pred_result['sequence'] = pd.to_numeric(pred_result['sequence'], errors='coerce').astype(pd.Int16Dtype())
pred_id = pd.read_csv('data/patents_2005_012.tsv', sep='\t', usecols=['id11', 'patent', 'fname', 'mname', 'lname', 'suffix'])
pred_id.drop_duplicates(inplace=True)

#join
pred = pd.merge(pred_result, pred_id, on=['patent', 'fname', 'mname', 'lname', 'suffix'])

#convert to series
pred.set_index('mention', inplace=True)
pred_series = pred['id11']
pred_series

mention
US6205043-1        1
US6434031-1        1
US6583702-1        1
US6423491-1        2
US4184768-1        3
               ...  
US6723088-1    17241
US6811612-4    28601
US6829498-3    40943
US6829498-4    41841
US6830895-1    49818
Name: id11, Length: 142400, dtype: int64

In [19]:
print("Predicted index duplicates:", pred_series.index.duplicated().sum())
print("Reference index duplicates:", ref_series.index.duplicated().sum())

Predicted index duplicates: 0
Reference index duplicates: 0


In [20]:
print(cluster_precision_recall(pred_series, ref_series))

(0.05977174980527617, 0.7391946025722117)
